# **TensorBoard for Visualization**
`TensorBoard` es una gran herramienta de visualización interactiva que
puedes usar para ver las curvas de aprendizaje durante el entrenamiento,
comparar curvas de aprendizaje entre múltiples ejecuciones, visualizar el
gráfico de computación, analizar estadísticas de entrenamiento, ver
imágenes generadas por tu modelo, visualizar datos multidimensionales
complejos proyectados hasta 3D y agrupados automáticamente ¡y
mucho más! Esta herramienta se instala automáticamente cuando
instalas TensorFlow


In [11]:
# Let's load, split and scale the California housing dataset(because SDG will be used)

from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

housing = fetch_california_housing()

X_train_full, X_test, y_train_full, y_test = train_test_split(housing.data, housing.target, random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)

En general, usted quiere apuntar el servidor TensorBoard a un directorio raíz de registro y
configurar su programa para que escriba en un subdirectorio diferente
cada vez que se ejecute para poder comprar distintps modeles entrenados sin que sean remplazados 

In [12]:
import os as os
root_logdir = os.path.join(os.curdir, "tensorboard_logs")


def get_run_logdir():
    import time
    run_id = time.strftime("run_%Y_%m_%d-%H_%M_%S")
    return os.path.join(root_logdir, run_id)


run_logdir = get_run_logdir()  # e.g., './my_logs/run_2019_06_07-15_15_22'


In [15]:
# build the model and compile

import tensorflow as tf
from tensorflow import keras
from keras.layers import Dense
from keras import Sequential
from keras.backend import clear_session
import numpy as np

# reiniciar (en caso de q ya se hayan ejecutados otras capas) la genracion del # de los sequential
clear_session()

# estableces las semillas para lograr reproducir los mismos resultados
np.random.seed(42)
tf.random.set_seed(42)

model = Sequential([
    Dense(30, activation="relu", input_shape=[8]),
    Dense(30, activation="relu"),
    Dense(1)
])
model.compile(loss="mse", optimizer=keras.optimizers.SGD(learning_rate=1e-3))


Keras provides a nice TensorBoard() callback

In [16]:
from keras.callbacks import TensorBoard
tensorboard_cb = TensorBoard(run_logdir)
history = model.fit(X_train, y_train, epochs=30,
                    validation_data=(X_valid, y_valid),
                    callbacks=[tensorboard_cb])


Epoch 1/30
363/363 [==============================] - 2s 4ms/step - loss: 0.8316 - val_loss: 7.8614
Epoch 2/30
363/363 [==============================] - 1s 3ms/step - loss: 0.5690 - val_loss: 0.6244
Epoch 3/30
363/363 [==============================] - 1s 3ms/step - loss: 0.4098 - val_loss: 0.5770
Epoch 4/30
363/363 [==============================] - 2s 4ms/step - loss: 0.3839 - val_loss: 0.3550
Epoch 5/30
363/363 [==============================] - 2s 5ms/step - loss: 0.3753 - val_loss: 0.3601
Epoch 6/30
363/363 [==============================] - 1s 4ms/step - loss: 0.3632 - val_loss: 0.3667
Epoch 7/30
363/363 [==============================] - 1s 3ms/step - loss: 0.3603 - val_loss: 0.3336
Epoch 8/30
363/363 [==============================] - 2s 5ms/step - loss: 0.3528 - val_loss: 0.3442
Epoch 9/30
363/363 [==============================] - 2s 4ms/step - loss: 0.3473 - val_loss: 0.3470
Epoch 10/30
363/363 [==============================] - 1s 3ms/step - loss: 0.3435 - val_loss: 0.3241

Ahora  falta iniciar el servidor de `TensorBoard` para tener las vizualizaciones 
```powershell
    tensorboard --logdir=./tensorboard_logs --port=6006
```
Alternativamente, puedes sustituir tensorboard en la línea de comandos por  
```
    python3 -m tensorboard.main
```

In [18]:
# usando los comandos inline de IPython
%load_ext tensorboard
%tensorboard --logdir=./tensorboard_logs --port=6006


The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


# **Fine-Tuning Neural Network Hyperparameters**

Una opción es simplemente probar muchas combinaciones de
hiperparámetros y ver cuál funciona mejor en el conjunto de validación (o
utilizar K -fold crossvalidation). Por ejemplo, podemos utilizar
**GridSearchCV** o **RandomizedSearchCV** para explorar el espacio de
hiperparámetros, y para eso necesitamos `wrappers(envoltorios)` de los modelos Keras para que imiten
regresores regulares de Scikit-Learn

In [20]:
clear_session()
np.random.seed(42)
tf.random.set_seed(42)

In [22]:
from keras.layers import InputLayer, Dense


def build_model(n_hidden=1, n_neurons=30, learning_rate=3e-3, input_shape=[8]):
    """create a function that will build and compile a Keras model
    """
    model = Sequential()
    model.add(InputLayer(input_shape=input_shape))
    for layer in range(n_hidden):
        model.add(Dense(n_neurons, activation="relu"))
    model.add(Dense(1))

    model.compile(loss="mse", optimizer=keras.optimizers.SGD(learning_rate=learning_rate))

    return model


In [24]:
# For we can use this object like a regular Scikit-Learn regressor: 
from keras.wrappers.scikit_learn import KerasRegressor
keras_reg = KerasRegressor(build_model) # DEPRECATED. Use Sci-Keras instead

C:\Users\Manue!_PC\AppData\Local\Temp\ipykernel_18020\3565169453.py:3: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  keras_reg = KerasRegressor(build_model) # DEPRECATED. Use Sci-Keras instead


In [25]:
# Cualquier parámetro extra que pases al método fit() se pasará al modelo subyacente de Keras
keras_reg.fit(X_train, y_train, epochs=100,
              validation_data=(X_valid, y_valid),
              callbacks=[keras.callbacks.EarlyStopping(patience=10)])

Epoch 1/100
363/363 [==============================] - 1s 2ms/step - loss: 1.0758 - val_loss: 10.5936
Epoch 2/100
363/363 [==============================] - 1s 2ms/step - loss: 0.6869 - val_loss: 2.1407
Epoch 3/100
363/363 [==============================] - 1s 2ms/step - loss: 0.5237 - val_loss: 0.4686
Epoch 4/100
363/363 [==============================] - 1s 2ms/step - loss: 0.4729 - val_loss: 0.4562
Epoch 5/100
363/363 [==============================] - 1s 2ms/step - loss: 0.4500 - val_loss: 0.4294
Epoch 6/100
363/363 [==============================] - 1s 2ms/step - loss: 0.4329 - val_loss: 0.4489
Epoch 7/100
363/363 [==============================] - 1s 2ms/step - loss: 0.4231 - val_loss: 0.4013
Epoch 8/100
363/363 [==============================] - 1s 2ms/step - loss: 0.4149 - val_loss: 0.3860
Epoch 9/100
363/363 [==============================] - 1s 2ms/step - loss: 0.4083 - val_loss: 0.4134
Epoch 10/100
363/363 [==============================] - 1s 2ms/step - loss: 0.4036 - val_l

In [26]:
mse_test = keras_reg.score(X_test, y_test)

162/162 [==============================] - 0s 1ms/step - loss: 0.3594


In [28]:
keras_reg.predict(X_test[:2])

1/1 [==============================] - 0s 32ms/step


array([0.5755028, 1.5916556], dtype=float32)

In [ ]:
np.random.seed(42)
tf.random.set_seed(42)

In [29]:
from scipy.stats import reciprocal
from sklearn.model_selection import RandomizedSearchCV
param_distribs = {
    "n_hidden": [0, 1, 2, 3],
    "n_neurons": np.arange(1, 100),
    "learning_rate": reciprocal(3e-4, 3e-2),
}
rnd_search_cv = RandomizedSearchCV(keras_reg, param_distribs, n_iter=10,
                                   cv=3)
rnd_search_cv.fit(X_train, y_train, epochs=100,
                  validation_data=(X_valid, y_valid),
                  callbacks=[keras.callbacks.EarlyStopping(patience=10)])


Epoch 1/100
242/242 [==============================] - 1s 2ms/step - loss: 3.1000 - val_loss: 1.3506
Epoch 2/100
242/242 [==============================] - 1s 2ms/step - loss: 0.9829 - val_loss: 0.7054
Epoch 3/100
242/242 [==============================] - 0s 2ms/step - loss: 0.6367 - val_loss: 0.6057
Epoch 4/100
242/242 [==============================] - 1s 2ms/step - loss: 0.5668 - val_loss: 0.5651
Epoch 5/100
242/242 [==============================] - 0s 2ms/step - loss: 0.5509 - val_loss: 0.5538
Epoch 6/100
242/242 [==============================] - 0s 2ms/step - loss: 0.5435 - val_loss: 0.7546
Epoch 7/100
242/242 [==============================] - 0s 2ms/step - loss: 0.5435 - val_loss: 0.6980
Epoch 8/100
242/242 [==============================] - 1s 2ms/step - loss: 0.5412 - val_loss: 0.7171
Epoch 9/100
242/242 [==============================] - 0s 2ms/step - loss: 0.5416 - val_loss: 0.5466
Epoch 10/100
242/242 [==============================] - 0s 2ms/step - loss: 0.5363 - val_lo

RandomizedSearchCV(cv=3,
                   estimator=<keras.wrappers.scikit_learn.KerasRegressor object at 0x000002C23172A320>,
                   param_distributions={'learning_rate': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000002C234D0B430>,
                                        'n_hidden': [0, 1, 2, 3],
                                        'n_neurons': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51,
       52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68,
       69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85,
       86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99])})

In [30]:
rnd_search_cv.best_params_

{'learning_rate': 0.0033625641252688094, 'n_hidden': 2, 'n_neurons': 42}

In [31]:
rnd_search_cv.best_score_

-0.3465402623017629

In [32]:
rnd_search_cv.score(X_test, y_test)

162/162 [==============================] - 0s 2ms/step - loss: 0.3534


-0.35335010290145874

In [33]:
model = rnd_search_cv.best_estimator_.model
model

In [34]:
model.evaluate(X_test, y_test)

162/162 [==============================] - 0s 2ms/step - loss: 0.3534


0.35335010290145874